/CT/
  ├── image/
  ├── nodule_mask/
  ├── metadata/
      ├── MetadatabyAnnotation.xlsx
      ├── AcqParams.xlsx
/models/
  └── malignancy_rf_model.pkl
/features/
  └── features.csv


In [1]:
!pip install SimpleITK


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 10.0 MB/s eta 0:00:00


In [2]:
!pip install radiomics

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.4 MB/s eta 0:00:00
  Created wheel for radiomics: filename=radiomics-0.1-py3-none-any.whl size=6090 sha256=0b9d0fe4fc2784ec8226a656d54f6294d4e21f0b22310b361f2e153cca417957
  Stored in directory: /root/.cache/pip/wheels/7b/d6/15/edae08f7c4249be729823f0c6b3c43abb9564dbcfa726b077a
Successfully built radiomics


In [3]:
!pip install pyradiomics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 25.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 41.5 MB/s eta 0:00:00
  Created wheel for pyradiomics: filename=

In [4]:
import os
import pandas as pd
import SimpleITK as sitk

# Path to your CT NIfTI images
input_dir = "/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/image"
output_file = "/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/image/AcqParams.xlsx"

# Default values for missing metadata (DICOM-derived)
DEFAULT_RESCALE_SLOPE = 1.0
DEFAULT_RESCALE_INTERCEPT = -1024
DEFAULT_WINDOW_CENTER = 40
DEFAULT_WINDOW_WIDTH = 400
DEFAULT_KVP = 120

records = []

# Iterate over all .nii.gz files
for filename in os.listdir(input_dir):
    if filename.endswith(".nii.gz"):
        filepath = os.path.join(input_dir, filename)
        img = sitk.ReadImage(filepath)
        spacing = img.GetSpacing()  # (x_spacing, y_spacing, z_spacing)

        record = {
            "PatientID": os.path.splitext(os.path.splitext(filename)[0])[0],
            "PixelSpacing[0]": spacing[0],
            "PixelSpacing[1]": spacing[1],
            "SliceThickness": spacing[2],
            "RescaleSlope": DEFAULT_RESCALE_SLOPE,
            "RescaleIntercept": DEFAULT_RESCALE_INTERCEPT,
            "WindowCenter": DEFAULT_WINDOW_CENTER,
            "WindowWidth": DEFAULT_WINDOW_WIDTH,
            "kVp": DEFAULT_KVP
        }
        records.append(record)

# Create DataFrame and save
df = pd.DataFrame(records)
df.to_excel(output_file, index=False)

print(f"Saved acquisition parameters to {output_file}")


Saved acquisition parameters to /content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/image/AcqParams.xlsx


In [ ]:
# 0. Setup & Imports
import os
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from skimage.filters import threshold_otsu
import SimpleITK as sitk
from radiomics import featureextractor
import warnings
warnings.filterwarnings("ignore")












In [ ]:
# ----------------------
# 1. Load Metadata & Parameters
# ----------------------
# Load MetadatabyAnnotation.xlsx and AcqParams.xlsx
metadata_df = pd.read_excel("/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/MetadatabyNoduleMaxVoting.xlsx")
acq_df = pd.read_excel("/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/image/AcqParams.xlsx")

# Display basic info
display(metadata_df.head())
display(acq_df.head())

In [ ]:
# ----------------------
# 2. Preprocessing Utilities
# ----------------------
def load_nifti(path):
    img = nib.load(path)
    return img.get_fdata(), img.affine, img.header

def convert_to_hu(image, slope, intercept):
    return image * slope + intercept

def resample(image, spacing, new_spacing=(1,1,1)):
    # Use SimpleITK for resampling
    return image  # placeholder

def normalize_ct(image, clip_range=(-1000, 400)):
    image = np.clip(image, *clip_range)
    return (image - clip_range[0]) / (clip_range[1] - clip_range[0])


In [ ]:
# ----------------------
# 3. Unsupervised Segmentation
# ----------------------
def segment_otsu(image):
    thresh = threshold_otsu(image)
    return image > thresh

def segment_kmeans(image, n_clusters=3):
    flat_img = image.reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters).fit(flat_img)
    clustered = kmeans.labels_.reshape(image.shape)
    return clustered == clustered[image.shape[0]//2, image.shape[1]//2, image.shape[2]//2]  # center

# Visualize a slice
def plot_slice(image, mask=None, slice_idx=None):
    if slice_idx is None:
        slice_idx = image.shape[2] // 2
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.imshow(image[:,:,slice_idx], cmap='gray')
    plt.title("CT Slice")
    if mask is not None:
        plt.subplot(1,2,2)
        plt.imshow(mask[:,:,slice_idx], cmap='gray')
        plt.title("Segmentation Mask")
    plt.show()

In [ ]:
# ----------------------
# 4. Feature Extraction
# ----------------------
def extract_features(image_path, mask_path):
    extractor = featureextractor.RadiomicsFeatureExtractor()
    result = extractor.execute(image_path, mask_path)
    return {k: result[k] for k in result if 'diagnostics' not in k}

In [ ]:
# ----------------------
# 5. Supervised Learning
# ----------------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assume X and y are ready
X = features_df.values
y = labels (0 = benign, 1 = malignant)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("ROC AUC:", roc_auc_score(y_test, pred))

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-24-09764c493fb6>, line 9)

In [ ]:
# ----------------------
# 6. Visualization & Explainability
# ----------------------
import shap
explainer = shap.Explainer(clf, X_train)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)

# ----------------------
# 7. Save/Export Results
# ----------------------
# Save model, results, or processed features
import joblib
joblib.dump(clf, "malignancy_rf_model.pkl")

In [ ]:
# Step 1: Feature Extraction with PyRadiomics
def extract_features(image_path, mask_path):
    # Initialize the feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()

    # Read the image and mask
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)

    # Extract features
    features = extractor.execute(image, mask)

    # Collect the feature values in a list
    feature_values = [features[key] for key in features.keys()]
    return feature_values

In [ ]:
# Step 2: Prepare the Dataset
def prepare_dataset(image_dir, mask_dir):
    # List to store features and labels
    features_list = []
    labels = []

    # Assuming the filenames of images and masks match in both directories
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz')])
    mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.nii.gz')])


    print(f"Found {len(image_files)} CT scan files")
    print(f"Found {len(mask_files)} Mask files")

    for image_file, mask_file in zip(image_files, mask_files):
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, mask_file)

        # Extract features
        features = extract_features(image_path, mask_path)
        features_list.append(features)

        # Assign label based on Malignancy annotation (dummy example here)
        # You need to implement your own label extraction logic, possibly from metadata.
        label = 1 if 'malignant' in image_file else 0  # Example: if filename contains 'malignant', set label as 1
        labels.append(label)

    # Convert features list to DataFrame
    X = pd.DataFrame(features_list)
    y = np.array(labels)

    return X, y

In [ ]:
# Function to train and evaluate the model
def train_and_evaluate(X, y):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Initialize a classifier (e.g., RandomForest)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the classifier
    clf.fit(X_train, y_train)

    # Predict on the test set
    y_pred = clf.predict(X_test)

    # Evaluate the model
    report = classification_report(y_test, y_pred)
    print("Classification Report:\n", report)

In [ ]:
# Function to process all CT scans and masks
def process_dataset(image_dir, mask_dir):
    features_list = []
    labels = []

    # Get list of CT scan and mask files
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz') or f.endswith('.nii')])
    mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.nii.gz') or f.endswith('.nii')])

    for image_file, mask_file in zip(image_files, mask_files):
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, mask_file)

        try:
            features = extract_features(image_path, mask_path)

            # If features are extracted, append them to the list
            if features:
                features_list.append(features)
                # Assign labels: You can modify this based on your dataset and annotation (e.g., 'malignant' or 'benign')
                label = 1 if 'malignant' in image_file else 0
                labels.append(label)
        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    return pd.DataFrame(features_list), np.array(labels)


In [ ]:
# Main pipeline function
def main(image_dir, mask_dir):
    # Process the dataset
    X, y = process_dataset(image_dir, mask_dir)

    if len(X) == 0:
        print("No features extracted. Please check your dataset and feature extraction.")
    else:
        print(f"Extracted {len(X)} samples with {X.shape[1]} features.")

        # Train and evaluate the model
        train_and_evaluate(X, y)

# Define your image and mask directories
image_dir = "/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/image"
mask_dir = "/content/drive/MyDrive/ML/Challenge2/LUNA Dataset /Full LUNA16 Dataset/VOIs/nodule_mask"

# Run the pipeline
main(image_dir, mask_dir)

Streaming output truncated to the last 5000 lines.
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomi